In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Embedding, Dense, BatchNormalization, Dropout, Input, Flatten
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import LabelEncoder

## Data preprocessing

In [3]:
def rebalance(frame, col='hab_lbl', factor=1):
    max_size = frame[col].value_counts().max()
    lst = [frame]
    for class_index, group in frame.groupby(col):
        lst.append(group.sample(int((max_size-len(group)) / factor), replace=True))
    frame_new = pd.concat(lst)
    
    return frame_new

Remove the "giveaway" column

In [4]:
df = pd.read_csv('dataset-rocky-no-STemp.csv')
df = df.drop('P. Habitable', axis=1)
#y = df['hab_lbl']
#df.drop('hab_lbl', axis=1, inplace=True)

In [5]:
df.shape

(1713, 42)

In [6]:
cat_vars = df.columns[np.where(df.dtypes == 'int64')]
cat_vars = cat_vars.tolist()
cat_vars.remove('hab_lbl')

In [7]:
cont_vars = df.columns[np.where(df.dtypes != 'int64')].tolist()

Encode the cateogrical variables.

In [8]:
label_encoders = {}

In [9]:
for cat_col in cat_vars:
    label_encoders[cat_col] = LabelEncoder()
    df[cat_col] = label_encoders[cat_col].fit_transform(df[cat_col])

In [10]:
cat_vars

['P. Zone Class',
 'P. Mass Class',
 'P. Composition Class',
 'P. Atmosphere Class']

In [7]:
train_df, test_df = train_test_split(df, train_size=0.7)

/home/rahul/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Rebalance only the training set.

In [8]:
train_df = rebalance(train_df)

In [9]:
train_df.shape

(3492, 42)

In [10]:
y = train_df['hab_lbl']

In [11]:
y_test = test_df['hab_lbl']

In [12]:
train_df.drop('hab_lbl', axis=1, inplace=True)
test_df.drop('hab_lbl', axis=1, inplace=True)

/home/rahul/.local/lib/python3.5/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
train_df.head()

,P. Zone Class,P. Mass Class,P. Composition Class,P. Atmosphere Class,P. Min Mass (EU),P. Mass (EU),P. Radius (EU),P. Density (EU),P. Gravity (EU),P. Esc Vel (EU),...,S. Appar Mag,S. Mag from Planet,S. Size from Planet (deg),S. Hab Zone Min (AU),S. Hab Zone Max (AU),P. HZD,P. HZC,P. HZA,P. HZI,P. ESI
752,2,4,3,3,18.217914,0.53,0.86,0.83,0.72,0.79,...,13.747447,-29.7,3.3814,0.286,0.707,-1.92,-0.18,-1.05,0.31,0.38
1154,2,2,3,1,18.217914,24.12,2.44,1.66,4.05,3.14,...,15.600000,-29.2,2.8653,0.301,0.748,-1.80,-0.11,1.15,0.32,0.36
252,2,4,3,2,18.217914,1.65,1.21,0.93,1.12,1.17,...,15.600000,-28.8,2.4066,0.268,0.665,-1.70,-0.17,-0.62,0.35,0.47
292,2,2,3,2,18.217914,18.31,2.30,1.51,3.47,2.82,...,13.747447,-32.5,6.6652,1.203,2.809,-2.35,-0.12,0.03,0.30,0.25
987,2,2,3,2,18.217914,5.54,1.70,1.12,1.91,1.80,...,14.800000,-32.2,5.8735,0.895,2.096,-2.32,-0.15,-0.56,0.29,0.28


Convert outputs to one-hot encoded outputs.

In [27]:
y = to_categorical(np.array(y))
y_test = to_categorical(np.array(y_test))

In [18]:
cat_inps = [list(train_df[x]) for x in cat_vars]
cont_inp = np.array(list(train_df[cont_vars]))

In [19]:
train_df[cont_vars].shape

(3495, 37)

In [20]:
y.shape

(3495, 3)

## Model

In [111]:
model_cat_inps = [Input(shape=(1,)) for _ in cat_inps]
model_cont_inp = Input(shape=(1, 37), name='cont_inp')

embeddings = [Embedding(input_dim=len(np.unique(x)),
                        output_dim=round(1.6 * len(np.unique(x)) ** 0.56)
                       )(y) for x, y in zip(cat_inps, model_cat_inps)]
bn1 = BatchNormalization(name='first_bn')(model_cont_inp)

concat = keras.layers.concatenate([*embeddings, bn1], name='concatenate')

relu = Dense(5, activation='relu', name='dense1')(concat)
bn = BatchNormalization(name='bn1')(relu)
drop = Dropout(0.2, name='dropout1')(bn)

relu = Dense(5, activation='relu', name='dense2')(drop)
bn = BatchNormalization()(relu)
drop = Dropout(0.2)(bn)

flat = Flatten()(drop)
out = Dense(3, activation='softmax', name='dense3')(flat)

model = Model(inputs=[*model_cat_inps, model_cont_inp], outputs=out)

In [112]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
cont_inp (

## LipschitzLR code

In [124]:
model.inputs

[<tf.Tensor 'input_5:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_6:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_7:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_8:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'cont_inp_1:0' shape=(?, 1, 37) dtype=float32>]

In [149]:
func = K.function(model.inputs, [model.layers[-2].output])

In [150]:
lrs = []

In [163]:
np.array(cat_inps).shape

(4, 3495)

In [17]:
x_train = np.array(train_df)
batch_size = 32

In [157]:
def lr_schedule(epoch, _):
    """Learning Rate Schedule
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    Kz = func([*cat_inps, np.array(train_df[cont_vars]).reshape(3495, 1, 37)])
    print(Kz)
    
    Kz = np.linalg.norm(Kz)

    K_ = (2. * Kz) / (3. * batch_size)
    lr = 1 / K_
    lrs.append(lr)
    print('Epoch', epoch, 'LR =', lr)
    return lr

In [158]:
lr_scheduler = LearningRateScheduler(lr_schedule)

## Train

In [159]:
model.compile(SGD(0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [160]:
len(cat_inps)

4

In [161]:
model.fit(x=[*cat_inps, np.array(train_df[cont_vars]).reshape(3495, 1, 37)], y=y, epochs=1, callbacks=[lr_scheduler])

Epoch 1/1


InvalidArgumentError: ConcatOp : Expected concatenating dimensions in the range [-2, 2), but got 2
	 [[{{node concatenate_1/concat}} = ConcatV2[N=5, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_5/embedding_lookup, embedding_6/embedding_lookup, embedding_7/embedding_lookup, embedding_8/embedding_lookup, first_bn_1/cond/Merge, concatenate_1/concat/axis)]]

In [105]:
model.inputs

[<tf.Tensor 'input_1:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_3:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'cont_inp:0' shape=(?, 1, 37) dtype=float32>]

In [117]:
model2 = keras.models.clone_model(model)

In [118]:
model2.layers.pop()

In [122]:
model2.layers

In [120]:
model2.predict(x=[*cat_inps, np.array(train_df[cont_vars]).reshape(3495, 1, 37)])

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [123]:
model2.predict(x=[*cat_inps, np.array(train_df[cont_vars]).reshape(3495, 1, 37)]).shape

(3495, 3)

## Alternative architecture

In [13]:
inp = Input(shape=(41,))

bn1 = BatchNormalization(name='first_bn')(inp)
relu = Dense(5, activation='relu', name='dense1')(bn1)
drop1 = Dropout(0.2, name='dropout1')(relu)

bn = BatchNormalization(name='bn1')(drop1)
relu = Dense(5, activation='relu', name='dense2')(bn)
drop2 = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop1, drop2])

bn = BatchNormalization(name='bn2')(interm)
relu = Dense(5, activation='relu', name='dense3')(bn)
drop = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop, drop2])

bn = BatchNormalization()(interm)
out = Dense(3, activation='softmax', name='dense4')(bn)

model = Model(inputs=inp, outputs=out)

In [14]:
func = K.function([model.layers[0].input], [model.layers[-2].output])

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 41)           0                                            
__________________________________________________________________________________________________
first_bn (BatchNormalization)   (None, 41)           164         input_1[0][0]                    
__________________________________________________________________________________________________
dense1 (Dense)                  (None, 5)            210         first_bn[0][0]                   
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, 5)            0           dense1[0][0]                     
__________________________________________________________________________________________________
bn1 (Batch

In [21]:
x_train = np.array(train_df)
batch_size = 32

In [18]:
np.linalg.norm(x_train)

1309925.3947118223

In [19]:
lrs = []

In [20]:
x_train.shape, y.shape

((3492, 41), (3492,))

In [22]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """

    Kz = 0.
    for i in range((len(x_train) - 1) // batch_size + 1):
        start_i = i * batch_size
        end_i = start_i + batch_size
        xb = x_train[start_i:end_i]
    
        activ = np.linalg.norm(func([xb]))
        if activ > Kz:
            Kz = activ

    K_ = (2. * Kz) / (3. * batch_size)
    lr = 1 / K_
    lrs.append(lr)
    print('Epoch', epoch, 'LR =', lr)
    return lr
    #return K_ / np.sqrt(epoch + 1)

In [23]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [24]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10)

Train on 3492 samples, validate on 514 samples
Epoch 1/10
3492/3492 [==============================] - 3s 887us/step - loss: 0.9644 - acc: 0.5444 - val_loss: 0.3834 - val_acc: 0.9611
Epoch 2/10
3492/3492 [==============================] - 1s 194us/step - loss: 0.6856 - acc: 0.6841 - val_loss: 0.2307 - val_acc: 0.9689
Epoch 3/10
3492/3492 [==============================] - 1s 198us/step - loss: 0.5589 - acc: 0.7646 - val_loss: 0.1823 - val_acc: 0.9747
Epoch 4/10
3492/3492 [==============================] - 1s 202us/step - loss: 0.5013 - acc: 0.7973 - val_loss: 0.1659 - val_acc: 0.9825
Epoch 5/10
3492/3492 [==============================] - 1s 196us/step - loss: 0.4567 - acc: 0.8204 - val_loss: 0.1422 - val_acc: 0.9844
Epoch 6/10
3492/3492 [==============================] - 1s 200us/step - loss: 0.4153 - acc: 0.8333 - val_loss: 0.1329 - val_acc: 0.9864
Epoch 7/10
3492/3492 [==============================] - 1s 194us/step - loss: 0.3591 - acc: 0.8634 - val_loss: 0.1245 - val_acc: 0.9864
E

In [29]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10, callbacks=[lr_scheduler])

Train on 3492 samples, validate on 514 samples
Epoch 1/10
Epoch 0 LR = 1.0273866637534177
3492/3492 [==============================] - 4s 1ms/step - loss: 0.3512 - acc: 0.8706 - val_loss: 0.8457 - val_acc: 0.7704
Epoch 2/10
Epoch 1 LR = 0.25070459981503956
3492/3492 [==============================] - 1s 258us/step - loss: 0.3301 - acc: 0.8731 - val_loss: 0.0901 - val_acc: 0.9864
Epoch 3/10
Epoch 2 LR = 0.41183196300523245
3492/3492 [==============================] - 1s 241us/step - loss: 0.2761 - acc: 0.8929 - val_loss: 0.0679 - val_acc: 0.9903
Epoch 4/10
Epoch 3 LR = 0.5928534254239075
3492/3492 [==============================] - 1s 269us/step - loss: 0.2332 - acc: 0.9135 - val_loss: 0.0815 - val_acc: 0.9864
Epoch 5/10
Epoch 4 LR = 0.49775331026265085
3492/3492 [==============================] - 1s 233us/step - loss: 0.1949 - acc: 0.9293 - val_loss: 0.0885 - val_acc: 0.9844
Epoch 6/10
Epoch 5 LR = 0.605234655008062
3492/3492 [==============================] - 1s 252us/step - loss: 0.2